# Advanced Tutorial 3 : build a Spectral Deferred Correction solver for non-linear ODEs

📜 _Previous base tutorial on [SDC](./04_sdc.ipynb) focused on the Dahlquist problem to explain how to use the_ $Q_\Delta$_-coefficients._
_But we can also use those for non-linear ODEs **as long as**_ $Q_\Delta$ _**is lower triangular**._

Back the **all-at-once system** defined for the [previous tutorial](./12_nonLinearRK.ipynb) :

$$
{\bf u} - \Delta{t}Q {\bf f} = {\bf u}_0
$$